# DriftWatch Advanced Customization

Learn how to customize thresholds, select specific detectors, and use the DriftExplainer.

In [ ]:
!pip install driftwatch[viz]

In [ ]:
import pandas as pd
import numpy as np

# Generate reference data (normal)
np.random.seed(42)
ref_data = pd.DataFrame({
    'age': np.random.normal(30, 5, 1000),
    'income': np.random.exponential(50000, 1000),
    'category': np.random.choice(['A', 'B', 'C'], 1000, p=[0.5, 0.3, 0.2])
})
ref_data.to_parquet('reference.parquet')

# Generate current data (drifted)
curr_data = pd.DataFrame({
    'age': np.random.normal(35, 5, 1000),  # Drifted mean
    'income': np.random.exponential(60000, 1000), # Drifted scale
    'category': np.random.choice(['A', 'B', 'C'], 1000, p=[0.3, 0.4, 0.3]) # Drifted probas
})
curr_data.to_parquet('current.parquet')
print('Data generated: reference.parquet, current.parquet')

### 1. Custom Thresholds
Stricter thresholds for critical features.

In [ ]:
from driftwatch import Monitor

# Initialize with a global stricter threshold
monitor = Monitor(reference_data=ref_data, default_threshold=0.01)

report = monitor.check(curr_data)
print(f"Drift Detected: {report.has_drift()}")
print(report)

### 2. Drift Explainer
Analyze why drift is happening using statistical properties.

In [ ]:
from driftwatch.explain import DriftExplainer

explainer = DriftExplainer(ref_data)
explanations = explainer.explain(curr_data)

# Look at 'age' explanation
print(explanations['age'])

### 3. Visualizing
Visualize the distribution shift.

In [ ]:
from driftwatch.explain import DriftVisualizer

viz = DriftVisualizer(ref_data)
viz.plot_feature(curr_data, 'age', title="Age Distribution Shift")